1. This notebook is based on the chatbot  which I build using the Langchain frame work.
3. The chatbot will use the data from your link.
4. I had used wikipedia as a example for the RAG process and looped the process of execution of the chat bot

In [ ]:
# Cell 1: Install required packages
!pip install langchain langchain_community langchain_groq requests beautifulsoup4 sentence-transformers langchain-huggingface


In [ ]:
# Cell 2: Verify installations
import importlib

In [ ]:
# Cell 2: Import necessary libraries and set up API key
import os
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

def check_import(module_name):
    try:
        importlib.import_module(module_name)
        print(f"{module_name} is installed and imported successfully.")
    except ImportError:
        print(f"Error: {module_name} is not installed or cannot be imported.")
        raise

# Check imports
check_import("langchain")
check_import("langchain_community")
check_import("langchain_groq")
check_import("requests")
check_import("bs4")
check_import("sentence_transformers")
check_import("langchain_huggingface")

langchain is installed and imported successfully.
langchain_community is installed and imported successfully.
langchain_groq is installed and imported successfully.
requests is installed and imported successfully.
bs4 is installed and imported successfully.


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


sentence_transformers is installed and imported successfully.
langchain_huggingface is installed and imported successfully.


In [ ]:
# Set your Groq API key
os.environ["GROQ_API_KEY"] = "Your_api_key "  #add you api key i had used the groq api for the reference

In [ ]:
# Cell 4: Set up the Groq LLM and define the data URL
llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0.7)

In [ ]:
# Define your custom data source URL
DATA_URL = "https://en.wikipedia.org/wiki/AlexNet#:~:text=AlexNet%20is%20the%20name%20of,D."  # Replace with your actual URL

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install langchain
!pip install faiss-cpu

from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Cell 6: Set up the retrieval system
loader = WebBaseLoader(DATA_URL) # Use WebBaseLoader from LangChain to load data from the URL
texts = loader.load()

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

<ipython-input-10-0cea2c8e8b48>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

In [ ]:

# Cell 5: Function to load and process data from the web page
def load_data(url):
    # Load web page data
    loader = WebBaseLoader(url)
    documents = loader.load()

    # Split text into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    return texts

In [ ]:
# Cell 6: Set up the retrieval system
texts = load_data(DATA_URL)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Cell 7: Create the conversational chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)


In [ ]:
# Cell 8: Chatbot interaction loop
chat_history = []

while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit', 'bye']:
        print("Bot: Goodbye!")
        break

    result = qa_chain({"question": user_input, "chat_history": chat_history})
    print(f"Bot: {result['answer']}")

    chat_history.append((user_input, result['answer']))

You: hi what is alex net
Bot: AlexNet is a convolutional neural network (CNN) architecture that was designed by Alex Krizhevsky in collaboration with Ilya Sutskever and Geoffrey Hinton. It was named after Alex Krizhevsky and was submitted to the ImageNet Large Scale Visual Recognition Challenge on September 30, 2012. The network achieved a top-5 error of 15.3%, which was more than 10.8 percentage points lower than that of the runner up. The architecture of AlexNet is known for its depth, which was made feasible due to the utilization of graphics processing units (GPUs) during training. The network consists of convolutional layers (with ReLU activation), local response normalization, maxpooling, fully connected layers (with ReLU activation), and dropout. The original AlexNet paper was published in 2012 and has been cited over 157,000 times as of mid-2024, making it one of the most influential papers in the field of computer vision.
You: exit
Bot: Goodbye!


In [19]:
# Cell 9: Accuracy measurement function

from sentence_transformers import SentenceTransformer, util
import numpy as np

# Initialize a sentence transformer model for computing similarity
similarity_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def compute_similarity(bot_answer, expected_answer):
    # Encode both answers
    bot_embedding = similarity_model.encode(bot_answer, convert_to_tensor=True)
    expected_embedding = similarity_model.encode(expected_answer, convert_to_tensor=True)

    # Compute cosine similarity
    similarity_score = util.pytorch_cos_sim(bot_embedding, expected_embedding).item()

    return similarity_score

def evaluate_accuracy(qa_chain, test_cases):
    """
    test_cases: A list of tuples containing question and expected answer (question, expected_answer)
    """
    correct_predictions = 0
    total_cases = len(test_cases)

    for question, expected_answer in test_cases:
        result = qa_chain({"question": question, "chat_history": []})
        bot_answer = result['answer']

        # Measure similarity between bot's answer and expected answer
        similarity_score = compute_similarity(bot_answer, expected_answer)
        print(f"Q: {question}")
        print(f"Bot: {bot_answer}")
        print(f"Expected: {expected_answer}")
        print(f"Similarity Score: {similarity_score}\n")

        # Assuming similarity above 0.8 is considered accurate
        if similarity_score > 0.8:
            correct_predictions += 1

    accuracy = correct_predictions / total_cases
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Example test cases (You can add more for a better evaluation)
test_cases = [
    ("was it first implemented with gpu ", "AlexNet was not the first fast GPU-implementation of a CNN to win an image recognition contest. A CNN on GPU by K. Chellapilla et al."),
    ("What is Alexnet", "AlexNet is the name of a convolutional neural network (CNN) architecture, designed by Alex Krizhevsky in collaboration with Ilya Sutskever and Geoffrey Hinton, who was Krizhevsky's Ph.D."),
    ("what is alexnet's Network design", "AlexNet contains eight layers: the first five are convolutional layers, some of them followed by max-pooling layers, and the last three are fully connected layers."),
]

# Call the evaluation function to measure accuracy
evaluate_accuracy(qa_chain, test_cases)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Q: was it first implemented with gpu 
Bot: No, AlexNet was not the first CNN to be implemented on a GPU. There were previous CNNs by K. Chellapilla et al. (2006) and Dan Cireșan et al. (2011) that were implemented on GPUs and outperformed CPU versions in terms of speed. AlexNet itself was originally written with CUDA to run with GPU support, following the CNN designs introduced by Yann LeCun et al. (1989) and later modified by J. Weng's method called max-pooling.
Expected: AlexNet was not the first fast GPU-implementation of a CNN to win an image recognition contest. A CNN on GPU by K. Chellapilla et al.
Similarity Score: 0.7757675647735596

Q: What is Alexnet
Bot: AlexNet is a convolutional neural network (CNN) architecture designed by Alex Krizhevsky, Ilya Sutskever, and Geoffrey Hinton. It is named after Alex Krizhevsky and was submitted to the ImageNet Large Scale Visual Recognition Challenge in 2012, where it achieved a top-5 error rate that was more than 10 percentage points lowe